In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import time

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
df_transport = pd.read_csv('./data/Dados_Onibus_Fortaleza.csv', delimiter=',', encoding='UTF-8',low_memory=False)

In [11]:
df_transport.head(3)

,linha,data_hora,validations_per_hour,d_semana,hour_sin,hour_cos,hora,d_mes,d_ano,mes,semana_do_mes,domingo,segunda,terca,quarta,quinta,sexta,sabado,feriado,vespera_feriado
0,1,2018-01-01 00:00:00,29,0,0.000000,1.000000,0,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,1,2018-01-01 01:00:00,58,0,0.269797,0.962917,1,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,1,2018-01-01 02:00:00,42,0,0.519584,0.854419,2,1,1,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [2]:
start_time = time.time()
# caminho_modelo = "mistralai/Mistral-7B-Instruct-v0.1"
# caminho_modelo = "TheBloke/30B-Epsilon-AWQ"
caminho_modelo = "TheBloke/laser-dolphin-mixtral-2x7b-dpo-AWQ"

In [3]:
system_message = f"""
    Faça a extração dos nomes de todas as pessoas com as suas respectivas cor de cabelo e altura. 
    Em hipótese alguma extraia outro tipo de informação, extraia apenas nome, cor do cabelo e altura.
    A sua saída deve retornar apenas um dicionário com os nomes das pessoas e suas respectivas informações.
    Por exemplo, para o texto: 
    "Alice é nadadora, mora em Fortaleza, tem um e cinquenta e possui cabelo loiro. Amanda é 15 centimetros mais alta que Alice e tem cabelo preto."
    A saída deve ser:
    Dicionário:
    {{
        "Alice": {{"cor_do_cabelo": "loiro", "altura": 1.5}},
        "Amanda": {{"cor_do_cabelo": "preto", "altura": 1.65}}
    }}
"""

In [4]:
user_message = f"""
    Agora, extraia as informações para o texto a seguir:

    "Carlos é programador e tem 1 metro e 69 centimentros. Alex é gerente de banco e é 5 centimentros mais baixo que Carlos. 
    Ambos têm cabelo castanho. Já Cláudio joga basquete, ele tem cabelo preto e possui 1.9 metros."
"""

In [5]:
if "MIXTRAL" in caminho_modelo.upper(): 
    prompt_template = f"""
        <|im_start|>system
        {system_message}
        <|im_end|>
        <|im_start|>user
        {user_message}
        <|im_end|>
        <|im_start|>assistant
        \n\n Dicionário:
    """

    response_splitter = "\n\n Dicionário:"

elif "ZEPHYR" in caminho_modelo.upper():

    prompt_template = f"""
        <|system|>
        {system_message}</s>
        <|user|>
        {user_message}</s>
        <|assistant|>
        \n\n Dicionário:
    """

    response_splitter = "\n\n Dicionário:"

elif "MISTRAL" in caminho_modelo.upper():

    prompt_template = f"""
        [INST]
        {system_message}
        
        {user_message}[/INST]
        \n\n Dicionário:
    """

    response_splitter = "\n\n Dicionário:"

elif "Epsilon".upper() in caminho_modelo.upper():

    prompt_template = f"""
        ### Instruction:
        {system_message}
        
        {user_message}
        ### Response:
        \n\n Dicionário:
    """

    response_splitter = "\n\n Dicionário:"

elif "TESS" in caminho_modelo.upper() or "WhiteRabbitNeo".upper() in caminho_modelo.upper():

    prompt_template = f"""
        SYSTEM:{system_message}        
        USER:{user_message}
        ASSISTANT:
        \n\n Dicionário:
    """

    response_splitter = "\n\n Dicionário:"

else:

    prompt_template = f"""
        {system_message}        
        {user_message}
        \n\n Dicionário:
    """

    response_splitter = "\n\n Dicionário:"

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == 'cuda':
    torch.set_default_dtype(torch.float16)

print(f"Using {device} device")

Using cpu device


In [8]:
tokenizer = AutoTokenizer.from_pretrained(caminho_modelo, trust_remote_code=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(caminho_modelo, device_map=device, trust_remote_code=True).to(device)

In [ ]:
# Codifica o prompt de entrada
tokens_entrada = tokenizer.encode(prompt_template, return_tensors='pt').to(device)

In [ ]:
max_length = tokens_entrada.shape[1] + 100  # Ajuste conforme necessário

In [ ]:
# Gera a máscara de atenção baseada nos tokens de entrada
attention_mask = torch.ones(tokens_entrada.shape, dtype=torch.long, device=device)  # Cria uma máscara de atenção onde todos os tokens são considerados (1s)

In [ ]:
# Certifica-se de que o pad_token_id esteja definido explicitamente se não estiver
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Gerar resumo com a máscara de atenção e pad_token_id definidos
dict = model.generate(
    input_ids=tokens_entrada,
    attention_mask=attention_mask,  # Passa a máscara de atenção
    max_length=max_length,
    do_sample=True,
    temperature=0.3,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id  # Garante que pad_token_id esteja definido explicitamente
)

In [ ]:
texto_entrada_saida = tokenizer.decode(dict[0], skip_special_tokens=True)

texto_saida = texto_entrada_saida.split(response_splitter)[1].strip()

In [ ]:
print('################## DICIONÁRIO ##################')
print(texto_saida)
print('################## DICIONÁRIO ##################')

In [ ]:
tokens_saida = tokenizer.encode(texto_saida, return_tensors='pt').to(device)

print(f"Quantidade de tokens na entrada: {tokens_entrada.shape[1]}")
print(f"Quantidade de tokens na saída: {tokens_saida.shape[1]}")

In [ ]:
end_time = time.time()
duration = end_time - start_time
print(f"Tempo de execução: {duration} segundos")